In [ ]:
# Colab: ランタイム起動直後に一度だけ
!pip -q install -U pip
!pip -q install -r env/requirements.txt "dvc[gcs]"


In [ ]:
from google.colab import auth
auth.authenticate_user()  # ブラウザで認可
print("✅ ADC 認証OK")


In [ ]:
from google.cloud import storage
client = storage.Client()  # ADC を使う
bucket_name = "<YOUR_BUCKET_NAME>"  # 例: rsna2025-<org>-prod
bucket = client.bucket(bucket_name)
print("✅ バケット存在:", bucket.exists())


In [ ]:
# 既に dvc.config.example がある前提。自分の環境ファイル dvc.config を作るならここで。

# 例: リモートは README 記載のとおり gs://.../rsna2025 を既定に
!cp -n dvc.config.example dvc.config || true
!dvc remote list
!dvc pull -q || echo "（初回は DVC で管理されているファイルが無い/権限不足の場合があります）"
print("✅ DVC pull 実行完了（メッセージを確認）")
